In [1]:
import numpy as np, pandas as pd, scipy.stats as st
from scipy.stats import t, norm
from joblib import Parallel, delayed

In [2]:
data = pd.read_csv('data.csv')
num_obvs = 100000
# risks per sector
r = np.array([.295, .49, .41, .415, .338, .64, .403, .476])
#sec_loading maps sector to its risk, t is the threshold for defaults
data['sec_loading'], data['t'] = r[data['sector'].values], st.norm.ppf(data.p)
# 100k monte carlo simulations and len(r)+len(data) risk factors per one sample
factors, sample, vars = np.random.normal(0,1, (num_obvs, len(r)+len(data))), [], []

In [3]:
for obs in factors:
    m_factor, sec_factor, res_factor = obs[0], obs[:len(r)][data.sector.values], obs[len(r):]
    ind = r[0]**.5 * m_factor + (data.sec_loading-r[0])**.5 * sec_factor + (1-
                                                                            data.sec_loading)**.5 * res_factor < data.t
    loss = np.zeros((len(data),))
    loss[ind] = data[ind].m + data[ind].d * np.random.standard_t(3, size=sum(ind))
    sample.append(sum(loss))
    vars.append(np.var(loss))


KeyboardInterrupt



In [ ]:
# extracts the 1% quantile from the sorted list of 100k obsercations
VaR = np.percentile(np.array(sample), 0.1)
variance = np.mean(vars)

In [ ]:
print("Średnia wariancji strat:", variance)
print("VaR antetyczny (100-ty wynik):", VaR)